# SimCLR Feature Extraction for Whole Slide Image Analysis

## Introduction

### What is SimCLR?

**SimCLR (Simple Framework for Contrastive Learning of Visual Representations)** is a self-supervised learning method developed by Google Research that learns meaningful visual features without requiring labeled data. Instead of relying on human annotations, SimCLR learns by comparing different augmented views of the same image.

### How SimCLR Works

The core idea is elegantly simple:

1. **Data Augmentation**: Take an image and create two different "views" by applying random augmentations (cropping, color jittering, flipping, blurring, etc.)

2. **Feature Extraction**: Pass both views through a neural network (ResNet-18 in our case) to get feature representations

3. **Contrastive Learning**: Train the network to recognize that these two views came from the *same* image, while distinguishing them from views of *other* images

4. **NT-Xent Loss**: The model uses Normalized Temperature-scaled Cross Entropy loss to pull together representations of the same image while pushing apart representations of different images

### Why SimCLR for Pathology?

Whole slide images (WSIs) present unique challenges:

- **Massive size**: A single WSI can be 100,000 × 100,000 pixels
- **Limited labels**: Expert pathologist annotations are expensive and time-consuming
- **High variability**: Staining differences, tissue artifacts, and scanner variations

SimCLR addresses these challenges by:

- Learning robust features from **unlabeled tissue patches**
- Capturing **histological patterns** (cellular structures, tissue architecture) without explicit supervision
- Producing features that **generalize well** to downstream tasks like cancer grading

### Our Pipeline

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│   Whole Slide   │     │   256×256 px    │     │    SimCLR       │
│     Images      │ ──▶ │   Tile Patches  │ ──▶ │   Training      │
│   (PANDA)       │     │   (JPEG tiles)  │     │   (ResNet-18)   │
└─────────────────┘     └─────────────────┘     └─────────────────┘
                                                        │
                                                        ▼
                                               ┌─────────────────┐
                                               │  512-dim Feature│
                                               │    Vectors      │
                                               └─────────────────┘
```

1. **Tiling**: WSIs are divided into 256×256 pixel patches at 20× magnification
2. **Self-Supervised Training**: SimCLR learns features from ~170,000 patches across ~8,600 training slides
3. **Feature Extraction**: The trained encoder produces a 512-dimensional feature vector for each patch
4. **Downstream Use**: These features feed into Graph Transformer networks (GTP) for slide-level cancer grading

### Training Configuration

| Parameter | Value |
|-----------|-------|
| Backbone | ResNet-18 |
| Feature Dimension | 512 |
| Batch Size | 76 |
| Epochs | 20 |
| Temperature (τ) | 0.5 |
| Optimizer | Adam |
| Learning Rate | 3e-4 (with cosine decay) |

### Key Insight

By pre-training on unlabeled pathology patches, SimCLR learns to identify meaningful tissue patterns—cell nuclei shapes, glandular structures, stromal patterns—that are directly relevant for cancer diagnosis. This self-supervised approach leverages the vast amount of unlabeled data available in digital pathology archives.

---

## Training Progress

The training loss (NT-Xent) decreases as the model learns to distinguish between different tissue patches:

- **Initial Loss**: ~6.0 (random features, poor discrimination)
- **Final Loss**: ~4.3 (learned features, good discrimination)

The decreasing loss indicates the model is successfully learning to:
1. Group similar tissue patterns together in feature space
2. Separate dissimilar patterns apart
3. Become invariant to augmentations (color shifts, rotations, etc.)

### Training SimCLR using ResNet backbone

In [13]:
from simclr import SimCLR
import yaml
from data_aug.dataset_wrapper import DataSetWrapper
import os, glob
import pandas as pd
import argparse
import sys
import gc

# REMOVE THIS LINE - let the scheduler handle GPU assignment
# os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'

def main():
    # Filter out ALL Jupyter kernel arguments
    sys.argv = [sys.argv[0]]  # Keep only the script name
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--magnification', type=str, default='20x')
    args = parser.parse_args()
    config = yaml.load(open("config.yaml", "r"), Loader=yaml.FullLoader)
    
    # Use the GPU(s) assigned by the scheduler
    # If you requested 1 GPU, use n_gpu=1
    config['n_gpu'] = 1
    config['gpu_ids'] = "[0]"  # The assigned GPU always appears as device 0
   
    dataset = DataSetWrapper(config['batch_size'], **config['dataset'])
    simclr = SimCLR(dataset, config)
    simclr.train()
    
if __name__ == "__main__":
    main()

Running on: cuda


AcceleratorError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
Search for `cudaErrorDevicesUnavailable' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


Test Model with weights using 10 random patches

In [7]:
#!/usr/bin/env python3
"""
Test SimCLR trained model - extract features from sample patches
"""
import torch
import torch.nn.functional as F
from models.resnet_simclr import ResNetSimCLR
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import os
from glob import glob

# ============================================================
# CONFIG
# ============================================================
CHECKPOINT_PATH = "runs/Nov29_15-00-34_scc-214/checkpoints/model.pth"
TILES_DIR = "/projectnb/ec500kb/projects/Project_1_Team_1/PANDA_DATA_MANNY/tiles_10"
NUM_TEST_PATCHES = 10  # Test on 10 random patches

# ============================================================
# LOAD MODEL
# ============================================================
def load_trained_model(checkpoint_path):
    """Load trained SimCLR model."""
    print("Loading trained SimCLR model...")
    
    # Initialize model (same as training)
    model = ResNetSimCLR(base_model="resnet18", out_dim=512)
    
    # Load trained weights
    state_dict = torch.load(checkpoint_path, map_location='cuda')
    
    # FIX: Remove 'module.' prefix from DataParallel
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] if k.startswith('module.') else k  # remove 'module.' prefix
        new_state_dict[name] = v
    
    model.load_state_dict(new_state_dict)
    
    # Set to evaluation mode
    model.eval()
    model = model.cuda()
    
    print(f"✓ Model loaded from {checkpoint_path}")
    return model

# ============================================================
# FEATURE EXTRACTION
# ============================================================
def extract_features(model, image_path):
    """Extract 512-dim features from a patch."""
    
    # Load and preprocess image
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).cuda()  # Add batch dimension
    
    # Extract features (no gradients needed)
    with torch.no_grad():
        h, z = model(img_tensor)  # h = features, z = projections
    
    return h.cpu().numpy(), z.cpu().numpy()


# ============================================================
# TEST
# ============================================================
def test_model():
    """Test the trained model on sample patches."""
    
    print("="*70)
    print("SIMCLR MODEL TEST")
    print("="*70)
    
    # Load model
    model = load_trained_model(CHECKPOINT_PATH)
    
    # Get sample patches
    print(f"\nFinding test patches in {TILES_DIR}...")
    all_patches = []
    for wsi_dir in os.listdir(TILES_DIR):
        wsi_path = os.path.join(TILES_DIR, wsi_dir)
        if os.path.isdir(wsi_path):
            patches = glob(os.path.join(wsi_path, "*.jpeg"))
            all_patches.extend(patches)
    
    # Sample random patches
    np.random.shuffle(all_patches)
    test_patches = all_patches[:NUM_TEST_PATCHES]
    
    print(f"✓ Found {len(all_patches)} total patches")
    print(f"✓ Testing on {len(test_patches)} random patches")
    print()
    
    # Extract features
    print("Extracting features...")
    print("-"*70)
    
    all_features = []
    all_projections = []
    
    for i, patch_path in enumerate(test_patches):
        patch_name = os.path.basename(patch_path)
        
        # Extract
        features, projections = extract_features(model, patch_path)
        all_features.append(features)
        all_projections.append(projections)
        
        # Print stats
        print(f"[{i+1}/{len(test_patches)}] {patch_name}")
        print(f"  Features shape:    {features.shape}")
        print(f"  Projections shape: {projections.shape}")
        print(f"  Feature norm:      {np.linalg.norm(features):.3f}")
        print(f"  Feature mean:      {features.mean():.3f} ± {features.std():.3f}")
        print()
    
    # Convert to arrays
    all_features = np.vstack(all_features)
    all_projections = np.vstack(all_projections)
    
    # Analysis
    print("="*70)
    print("FEATURE ANALYSIS")
    print("="*70)
    print(f"Features shape:    {all_features.shape}")
    print(f"Projections shape: {all_projections.shape}")
    print()
    
    print("Feature statistics:")
    print(f"  Mean:     {all_features.mean():.4f}")
    print(f"  Std:      {all_features.std():.4f}")
    print(f"  Min:      {all_features.min():.4f}")
    print(f"  Max:      {all_features.max():.4f}")
    print()
    
    # Test similarity (patches should have diverse features)
    print("Feature diversity check:")
    from scipy.spatial.distance import cosine
    
    similarities = []
    for i in range(len(all_features)):
        for j in range(i+1, len(all_features)):
            sim = 1 - cosine(all_features[i], all_features[j])
            similarities.append(sim)
    
    similarities = np.array(similarities)
    print(f"  Pairwise cosine similarity:")
    print(f"    Mean: {similarities.mean():.3f}")
    print(f"    Std:  {similarities.std():.3f}")
    print(f"    Min:  {similarities.min():.3f}")
    print(f"    Max:  {similarities.max():.3f}")
    print()
    
    # Interpretation
    print("="*70)
    print("INTERPRETATION")
    print("="*70)
    
    if similarities.mean() < 0.5:
        print("✓ GOOD: Features are diverse (low similarity)")
        print("  → Model learned to distinguish different patches")
    elif similarities.mean() < 0.8:
        print("✓ OK: Features have moderate diversity")
        print("  → Model provides useful representations")
    else:
        print("⚠ WARNING: Features are very similar")
        print("  → Model may not have learned well")
    
    print()
    print("Model is ready to use for feature extraction! ✓")
    print("="*70)


# ============================================================
# MAIN
# ============================================================
if __name__ == "__main__":
    test_model()


SIMCLR MODEL TEST
Loading trained SimCLR model...
Feature extractor: resnet18
✓ Model loaded from runs/Nov29_15-00-34_scc-214/checkpoints/model.pth

Finding test patches in /projectnb/ec500kb/projects/Project_1_Team_1/PANDA_DATA_MANNY/tiles_10...
✓ Found 121860 total patches
✓ Testing on 10 random patches

Extracting features...
----------------------------------------------------------------------
[1/10] 11_56.jpeg
  Features shape:    (512,)
  Projections shape: (512,)
  Feature norm:      25.359
  Feature mean:      0.577 ± 0.961

[2/10] 13_16.jpeg
  Features shape:    (512,)
  Projections shape: (512,)
  Feature norm:      24.897
  Feature mean:      0.613 ± 0.914

[3/10] 45_4.jpeg
  Features shape:    (512,)
  Projections shape: (512,)
  Feature norm:      27.561
  Feature mean:      0.819 ± 0.901

[4/10] 7_43.jpeg
  Features shape:    (512,)
  Projections shape: (512,)
  Feature norm:      22.274
  Feature mean:      0.531 ± 0.829

[5/10] 42_23.jpeg
  Features shape:    (512,)
  

In [8]:
#!/usr/bin/env python3
"""
Test ImageNet pretrained ResNet - extract features from sample patches
"""
import torch
import torch.nn.functional as F
import torchvision.models as models
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import os
from glob import glob

# ============================================================
# CONFIG
# ============================================================
TILES_DIR = "/projectnb/ec500kb/projects/Project_1_Team_1/PANDA_DATA_MANNY/tiles_10"
NUM_TEST_PATCHES = 10  # Test on 10 random patches
USE_RESNET50 = True  # True for ResNet50 (2048-dim), False for ResNet18 (512-dim)

# ============================================================
# LOAD MODEL
# ============================================================
def load_imagenet_model(use_resnet50=True):
    """Load ImageNet pretrained ResNet model."""
    print("Loading ImageNet pretrained ResNet model...")
    
    if use_resnet50:
        print("Feature extractor: resnet50")
        # ResNet50 - gives 2048-dim features
        model = models.resnet50(weights='IMAGENET1K_V1')
        feature_dim = 2048
    else:
        print("Feature extractor: resnet18")
        # ResNet18 - gives 512-dim features (same as your SimCLR)
        model = models.resnet18(weights='IMAGENET1K_V1')
        feature_dim = 512
    
    # Remove the final classification layer (FC layer)
    # Keep everything up to avgpool
    model = torch.nn.Sequential(*list(model.children())[:-1])
    
    # Set to evaluation mode
    model.eval()
    model = model.cuda()
    
    print(f"✓ ImageNet pretrained model loaded")
    print(f"✓ Feature dimension: {feature_dim}")
    return model, feature_dim

# ============================================================
# FEATURE EXTRACTION
# ============================================================
def extract_features(model, image_path):
    """Extract features from a patch using ImageNet pretrained model."""
    
    # ImageNet normalization (IMPORTANT!)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                           std=[0.229, 0.224, 0.225])
    ])
    
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).cuda()  # Add batch dimension
    
    # Extract features (no gradients needed)
    with torch.no_grad():
        features = model(img_tensor)  # Shape: [1, feature_dim, 1, 1]
        features = features.squeeze()  # Shape: [feature_dim]
    
    return features.cpu().numpy()


# ============================================================
# TEST
# ============================================================
def test_model():
    """Test the ImageNet pretrained model on sample patches."""
    
    print("="*70)
    print("IMAGENET PRETRAINED RESNET TEST")
    print("="*70)
    
    # Load model
    model, feature_dim = load_imagenet_model(use_resnet50=USE_RESNET50)
    
    # Get sample patches
    print(f"\nFinding test patches in {TILES_DIR}...")
    all_patches = []
    for wsi_dir in os.listdir(TILES_DIR):
        wsi_path = os.path.join(TILES_DIR, wsi_dir)
        if os.path.isdir(wsi_path):
            patches = glob(os.path.join(wsi_path, "*.jpeg"))
            all_patches.extend(patches)
    
    # Sample random patches
    np.random.shuffle(all_patches)
    test_patches = all_patches[:NUM_TEST_PATCHES]
    
    print(f"✓ Found {len(all_patches)} total patches")
    print(f"✓ Testing on {len(test_patches)} random patches")
    print()
    
    # Extract features
    print("Extracting features...")
    print("-"*70)
    
    all_features = []
    
    for i, patch_path in enumerate(test_patches):
        patch_name = os.path.basename(patch_path)
        
        # Extract
        features = extract_features(model, patch_path)
        all_features.append(features)
        
        # Print stats
        print(f"[{i+1}/{len(test_patches)}] {patch_name}")
        print(f"  Features shape:    {features.shape}")
        print(f"  Feature norm:      {np.linalg.norm(features):.3f}")
        print(f"  Feature mean:      {features.mean():.3f} ± {features.std():.3f}")
        print()
    
    # Convert to array
    all_features = np.vstack(all_features)
    
    # Analysis
    print("="*70)
    print("FEATURE ANALYSIS")
    print("="*70)
    print(f"Features shape:    {all_features.shape}")
    print()
    
    print("Feature statistics:")
    print(f"  Mean:     {all_features.mean():.4f}")
    print(f"  Std:      {all_features.std():.4f}")
    print(f"  Min:      {all_features.min():.4f}")
    print(f"  Max:      {all_features.max():.4f}")
    print()
    
    # Test similarity (patches should have diverse features)
    print("Feature diversity check:")
    from scipy.spatial.distance import cosine
    
    similarities = []
    for i in range(len(all_features)):
        for j in range(i+1, len(all_features)):
            sim = 1 - cosine(all_features[i], all_features[j])
            similarities.append(sim)
    
    similarities = np.array(similarities)
    print(f"  Pairwise cosine similarity:")
    print(f"    Mean: {similarities.mean():.3f}")
    print(f"    Std:  {similarities.std():.3f}")
    print(f"    Min:  {similarities.min():.3f}")
    print(f"    Max:  {similarities.max():.3f}")
    print()
    
    # Interpretation
    print("="*70)
    print("INTERPRETATION")
    print("="*70)
    
    if similarities.mean() < 0.5:
        print("✓ GOOD: Features are diverse (low similarity)")
        print("  → Model learned to distinguish different patches")
    elif similarities.mean() < 0.8:
        print("✓ OK: Features have moderate diversity")
        print("  → Model provides useful representations")
    else:
        print("⚠ WARNING: Features are very similar")
        print("  → Model may not have learned well")
    
    print()
    print("Model is ready to use for feature extraction! ✓")
    print("="*70)
    print()
    print("COMPARISON WITH YOUR SIMCLR:")
    print(f"  SimCLR features:  512-dim")
    print(f"  ImageNet features: {feature_dim}-dim")
    print(f"  ImageNet uses standard normalization and is pretrained on 1.2M images")
    print("="*70)


# ============================================================
# MAIN
# ============================================================
if __name__ == "__main__":
    test_model()

IMAGENET PRETRAINED RESNET TEST
Loading ImageNet pretrained ResNet model...
Feature extractor: resnet50
✓ ImageNet pretrained model loaded
✓ Feature dimension: 2048

Finding test patches in /projectnb/ec500kb/projects/Project_1_Team_1/PANDA_DATA_MANNY/tiles_10...
✓ Found 121860 total patches
✓ Testing on 10 random patches

Extracting features...
----------------------------------------------------------------------
[1/10] 20_28.jpeg
  Features shape:    (2048,)
  Feature norm:      20.777
  Feature mean:      0.292 ± 0.355

[2/10] 1_27.jpeg
  Features shape:    (2048,)
  Feature norm:      24.013
  Feature mean:      0.353 ± 0.396

[3/10] 1_9.jpeg
  Features shape:    (2048,)
  Feature norm:      22.675
  Feature mean:      0.328 ± 0.379

[4/10] 11_29.jpeg
  Features shape:    (2048,)
  Feature norm:      29.991
  Feature mean:      0.446 ± 0.490

[5/10] 37_32.jpeg
  Features shape:    (2048,)
  Feature norm:      28.593
  Feature mean:      0.448 ± 0.446

[6/10] 18_11.jpeg
  Features 

In [15]:
import torch
import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
from tqdm import tqdm
from collections import OrderedDict

class TileDataset(Dataset):
    def __init__(self, tile_base_dir, label_file, max_slides=50, tiles_per_slide=40, max_samples=2000):
        """Load tiles with REAL labels from val_set.txt"""
        self.tiles = []
        self.labels = []
        
        # Load labels from val_set.txt
        print("Loading labels...")
        label_dict = {}
        with open(label_file, 'r') as f:
            for line in f:
                if line.strip():
                    parts = line.strip().split()
                    if len(parts) == 2:
                        slide_id = parts[0].split('/')[-1]  # Extract ID after 'panda/'
                        label = int(parts[1])
                        label_dict[slide_id] = label
        
        print(f"✓ Loaded {len(label_dict)} slide labels")
        print(f"  Class distribution: {np.bincount(list(label_dict.values()))}")
        
        # Get slide directories
        slide_dirs = [d for d in os.listdir(tile_base_dir) 
                     if os.path.isdir(os.path.join(tile_base_dir, d))]
        
        # Filter to slides with labels
        labeled_slides = [s for s in slide_dirs if s in label_dict][:max_slides]
        print(f"\nLoading tiles from {len(labeled_slides)} slides...")
        
        for slide_id in tqdm(labeled_slides):
            label = label_dict[slide_id]
            slide_path = os.path.join(tile_base_dir, slide_id)
            
            # Get tile files
            tile_files = [f for f in os.listdir(slide_path) 
                         if f.endswith('.jpeg') or f.endswith('.jpg')][:tiles_per_slide]
            
            if len(tile_files) == 0:
                continue
            
            # Add tiles with REAL labels
            for tile_file in tile_files:
                self.tiles.append(os.path.join(slide_path, tile_file))
                self.labels.append(label)
            
            if len(self.tiles) >= max_samples:
                break
        
        self.tiles = self.tiles[:max_samples]
        self.labels = self.labels[:max_samples]
        
        print(f"✓ Loaded {len(self.tiles)} tiles")
        print(f"  Class distribution: {np.bincount(self.labels)}")
    
    def __len__(self):
        return len(self.tiles)
    
    def __getitem__(self, idx):
        img = Image.open(self.tiles[idx]).convert('RGB')
        img = img.resize((224, 224))
        img = torch.from_numpy(np.array(img)).permute(2, 0, 1).float() / 255.0
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        img = (img - mean) / std
        return img, self.labels[idx]


print("="*70)
print("SIMCLR LINEAR PROBE TEST (WITH REAL LABELS)")
print("="*70)

# Load SimCLR model
from models.resnet_simclr import ResNetSimCLR

model = ResNetSimCLR('resnet18', out_dim=512)
checkpoint = torch.load('runs/Nov29_15-00-34_scc-214/checkpoints/model.pth', 
                       map_location='cuda')

new_checkpoint = OrderedDict()
for k, v in checkpoint.items():
    new_checkpoint[k.replace('module.', '')] = v

model.load_state_dict(new_checkpoint)
model = model.cuda().eval()
print("✓ SimCLR model loaded\n")

# Load tiles WITH REAL LABELS
tile_dir = '/projectnb/ec500kb/projects/Project_1_Team_1/PANDA_DATA_MANNY/val_tiles'
label_file = '/projectnb/ec500kb/projects/Project_1_Team_1/Official_GTP_PANDAS/PANDAS/scripts/val_set.txt'

dataset = TileDataset(tile_dir, label_file, max_slides=50, tiles_per_slide=40, max_samples=2000)
loader = DataLoader(dataset, batch_size=64, num_workers=4, shuffle=False)

# Extract features
print("\nExtracting features...")
features_list = []
labels_list = []

with torch.no_grad():
    for imgs, lbls in tqdm(loader):
        feats = model.features(imgs.cuda())
        feats = torch.flatten(feats, start_dim=1)
        features_list.append(feats.cpu().numpy())
        labels_list.extend(lbls.numpy())

features = np.vstack(features_list)
labels = np.array(labels_list)

print(f"\n✓ Extracted features: {features.shape}")
print(f"  Classes: {np.unique(labels)}")
print(f"  Class distribution: {np.bincount(labels)}")

# Train/test split
split = int(0.8 * len(features))
indices = np.random.permutation(len(features))
train_idx = indices[:split]
test_idx = indices[split:]

X_train, X_test = features[train_idx], features[test_idx]
y_train, y_test = labels[train_idx], labels[test_idx]

# Linear probe
print("\nTraining linear classifier...")
clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(X_train, y_train)

train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)

# Per-class accuracy
y_pred = clf.predict(X_test)
from sklearn.metrics import confusion_matrix, classification_report

print("\n" + "="*70)
print("LINEAR PROBE RESULTS")
print("="*70)
print(f"Train samples: {len(X_train)}")
print(f"Test samples:  {len(X_test)}")
print(f"\nTrain accuracy: {train_acc:.3f} ({train_acc*100:.1f}%)")
print(f"Test accuracy:  {test_acc:.3f} ({test_acc*100:.1f}%)")

print(f"\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

print(f"\nPer-class Report:")
print(classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1', 'Class 2']))

print(f"\n{'='*70}")
print("INTERPRETATION")
print("="*70)
print(f"Random baseline (3 classes): 33.3%")
print(f"Your model: {test_acc*100:.1f}%")

if test_acc < 0.40:
    print("\n✗ POOR - Worse than random!")
    print("  SimCLR failed completely")
elif test_acc < 0.50:
    print("\n⚠ WEAK - Barely better than random")
    print("  Features have minimal discriminative power")
elif test_acc < 0.60:
    print("\n⚠ MEDIOCRE - Features are weak")
    print("  Usable but not great")
elif test_acc < 0.70:
    print("\n✓ GOOD - Features are working well")
    print("  SimCLR learned meaningful representations")
else:
    print("\n✓✓ EXCELLENT - Strong features")
    print("  SimCLR is very effective")

print("="*70)

SIMCLR LINEAR PROBE TEST (WITH REAL LABELS)
Feature extractor: resnet18
✓ SimCLR model loaded

Loading labels...
✓ Loaded 2123 slide labels
  Class distribution: [ 568  486 1069]

Loading tiles from 50 slides...


 98%|█████████▊| 49/50 [00:00<00:00, 6346.76it/s]


✓ Loaded 2000 tiles
  Class distribution: [520 560 920]

Extracting features...


100%|██████████| 32/32 [00:02<00:00, 14.62it/s]



✓ Extracted features: (2000, 512)
  Classes: [0 1 2]
  Class distribution: [520 560 920]

Training linear classifier...

LINEAR PROBE RESULTS
Train samples: 1600
Test samples:  400

Train accuracy: 0.774 (77.4%)
Test accuracy:  0.625 (62.5%)

Confusion Matrix:
[[ 65  17  29]
 [ 25  34  47]
 [  7  25 151]]

Per-class Report:
              precision    recall  f1-score   support

     Class 0       0.67      0.59      0.62       111
     Class 1       0.45      0.32      0.37       106
     Class 2       0.67      0.83      0.74       183

    accuracy                           0.62       400
   macro avg       0.59      0.58      0.58       400
weighted avg       0.61      0.62      0.61       400


INTERPRETATION
Random baseline (3 classes): 33.3%
Your model: 62.5%

✓ GOOD - Features are working well
  SimCLR learned meaningful representations


/projectnb/ec500kb/projects/Project_1_Team_1/panda_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
# 1. What's in your val_set.txt?
!head -20 /projectnb/ec500kb/projects/Project_1_Team_1/Official_GTP_PANDAS/PANDAS/scripts/val_set.txt

# 2. Do you have the original PANDA data?
!ls /projectnb/ec500kb/projects/Project_1_Team_1/PANDA_DATA_MANNY/

# 3. Look for CSV files with labels
!find /projectnb/ec500kb/projects/Project_1_Team_1/ -name "*.csv" | grep -i label
!find /projectnb/ec500kb/projects/Project_1_Team_1/ -name "*.csv" | grep -i train

# 4. Check PANDA original files
!ls /projectnb/ec500kb/projects/Project_1_Team_1/PANDA_DATA_MANNY/ | grep csv

a


ModuleNotFoundError: No module named 'utils'